In [1]:
import pandas as pd
import requests
import time
import numpy as np
import glob
import bs4
import pypyodbc as podbc
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
#pandas params
pd.set_option('display.max_rows', None)
pd.set_option('display.min_rows', 200)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

We're going to set a couple of variables that can be changed to reflect the current season and date throughout the script to prevent having to manually change things. 

In [3]:
this_year = '2019-20'
last_year = '2018-19'
todays_date = '2020-12-22'

# Scraping data

### # Per-game values

Now we'll build a function to scrape per-game values for a given season. In practice, we will only be scraping the *current* seaon's per-game values. 

In [4]:
#Function to scrape per-game values
def scrape_per_game(season):
    headers = {
'Host': 'stats.nba.com',
'Connection': 'keep-alive',
'Accept': 'application/json, text/plain, */*',
'x-nba-stats-token': 'true',
'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36',
'x-nba-stats-origin': 'stats',
'Referer': 'https://www.nba.com/',
'Accept-Encoding': 'gzip, deflate, br',
'Accept-Language': 'en-US,en;q=0.9'}

    url = f"https://stats.nba.com/stats/leaguedashplayerstats?College=&Conference=&Country=&DateFrom=&DateTo=&Division=&DraftPick=&DraftYear=&GameScope=&GameSegment=&Height=&LastNGames=0&LeagueID=00&Location=&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PaceAdjust=N&PerMode=PerGame&Period=0&PlayerExperience=&PlayerPosition=&PlusMinus=N&Rank=N&Season={season}&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&StarterBench=&TeamID=0&TwoWay=0&VsConference=&VsDivision=&Weight="
    r = requests.get(url, headers=headers).json()

    df = pd.DataFrame(r['resultSets'][0]['rowSet'], columns = r['resultSets'][0]['headers'])
    df['Season'] = season
    return df

Create per_game, a DataFrame to hold per-game values for relevant seasons

In [5]:
#First, we pull in the per-game values from prior seasons
past = pd.read_csv("C:\\Users\gsteele\Other\per_game_past.csv")

#Temporary, will NOT go into production
past = past[past['Season'] != '2019-20']

current = scrape_per_game(this_year)
frame_list = [past,current]
per_game = pd.concat(frame_list)

#Calculate two-pointers
per_game.insert(loc = 16, column = 'FG2M', value = (per_game.FGM - per_game.FG3M))
per_game.insert(loc = 17, column = 'FG2A', value = (per_game.FGA - per_game.FG3A))

#DraftKings average
per_game.insert(loc = 1, column = 'draftkings', value = (
        (per_game.FG3M*3.5)+(per_game.FG2M*2)+(per_game.FTM)+(per_game.REB*1.25)+(per_game.BLK*2)+(per_game.STL*2)
        +(per_game.TOV*(-0.5))+(per_game.AST*1.5)
        )
)
#FanDuel average
per_game.insert(loc = 1, column = 'fanduel', value = (
        (per_game.FG3M*3)+(per_game.FG2M*2)+(per_game.FTM)+(per_game.REB*1.2)+(per_game.BLK*2)+(per_game.STL*2)
        +(per_game.TOV*(-1))+(per_game.AST*1.5)
        )
)

per_game = per_game[['PLAYER_ID','PLAYER_NAME','Season','fanduel','draftkings','TEAM_ID','TEAM_ABBREVIATION','GP','MIN','PTS','FGM',
                     'FGA','FG2M','FG2A','FG3M','FG3A','FG_PCT','FG3_PCT','FTM','FTA','FT_PCT','OREB','DREB','REB','AST','TOV',
                    'STL','BLK','PF','PFD','DD2','TD3']]

### Update matchup data

The following function pulls down matchup data for a single in the current season, then concatenates into a single DataFrame

In [6]:
def scrape_matchup_data(teamid):
    headers = {
'Host': 'stats.nba.com',
'Connection': 'keep-alive',
'Accept': 'application/json, text/plain, */*',
'x-nba-stats-token': 'true',
'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36',
'x-nba-stats-origin': 'stats',
'Referer': f"https://stats.nba.com/team/{teamid}/matchups/",
'Accept-Encoding': 'gzip, deflate, br',
'Accept-Language': 'en-US,en;q=0.9'}

    url = f"https://stats.nba.com/stats/leagueseasonmatchups?DateFrom=&DateTo=&DefTeamID={teamid}&LeagueID=00&Outcome=&PORound=0&PerMode=Totals&Season=2019-20&SeasonType=Regular+Season"""
    r = requests.get(url, headers=headers).json()
    df = pd.DataFrame(r['resultSets'][0]['rowSet'], columns = r['resultSets'][0]['headers'])
    return df

csv_list = []
for team_id in range (1610612737,1610612768):
    time.sleep(np.random.randint(0,5 + 1))
    frame = scrape_matchup_data(team_id)
    csv_list.append(frame)
    
new_matchups = pd.concat(csv_list)

Next, we combine the current season's matchup data with previous data

In [7]:
#Import matchup data
old_matchups = pd.read_csv(r'c:\users\gsteele\Other\matchup_data.csv', low_memory = False)

#The next line will be deleted BEFORE PRODUCTION
old_matchups = old_matchups[old_matchups['SEASON_ID'] != '22019']

#Combine current season with previous data
matchups = [new_matchups,old_matchups]
matchups_df = pd.concat(matchups)

#Handle season naming conventions
matchups_df.loc[matchups_df['SEASON_ID'] == '22017', 'SEASON_ID'] = '2017-18'
matchups_df.loc[matchups_df['SEASON_ID'] == '22018', 'SEASON_ID'] = '2018-19'
matchups_df.loc[matchups_df['SEASON_ID'] == '22019', 'SEASON_ID'] = '2019-20'

#Calculate FG2M and FG2A
matchups_df.insert(loc = 17, column = 'MATCHUP_FG2M', value = (matchups_df['MATCHUP_FGM'] - matchups_df['MATCHUP_FG3M']))
matchups_df.insert(loc = 17, column = 'MATCHUP_FG2A', value = (matchups_df['MATCHUP_FGA'] - matchups_df['MATCHUP_FG3M']))

# Transform data

### Matchups2

Our first step will be to join matchup data to the per-game data. We'll also aggregate total possessions and possessions per game within a season for each player ("offesnive" player). We're also going to round up PARTIAL_POSS values below 0.5 up to 0.5.

In [8]:
#Join per-game with matchup data
matchups2 = pd.merge(left = per_game, right = matchups_df, 
                     how = 'left', left_on = ['PLAYER_ID','Season'], right_on = ['OFF_PLAYER_ID','SEASON_ID'])

#Determining total possessions and possessions per game for offensive player
matchups2.insert(loc = 7, column = 'total_possessions', value = 
                 (matchups2.groupby(['PLAYER_ID','Season'])['PARTIAL_POSS'].transform('sum')
                 )
                )

matchups2.insert(loc = 8, column = 'poss_per_game', value = (matchups2['total_possessions']/(matchups2['GP_x'])))

#rounding up values below a half-possession for a game
matchups2['PARTIAL_POSS'].update(np.where(matchups2['PARTIAL_POSS']<0.5, 0.5, matchups2['PARTIAL_POSS']))

Now tat we know how many possessions per game each player plays, we can calculate their average per-possession vales for the given season. 

In [9]:
#Calculate per-possession values for offensive player
matchups2.insert(loc = 9, column = 'fanduel_poss', value = (matchups2['fanduel']/matchups2['poss_per_game']))
matchups2.insert(loc = 10, column = 'draftkings_poss', value = (matchups2['draftkings']/matchups2['poss_per_game']))
matchups2.insert(loc = 11, column = 'PTS_poss', value = (matchups2['PTS']/matchups2['poss_per_game']))
matchups2.insert(loc = 12, column = 'FGM_poss', value = (matchups2['FGM']/matchups2['poss_per_game']))
matchups2.insert(loc = 13, column = 'FGA_poss', value = (matchups2['FGA']/matchups2['poss_per_game']))
matchups2.insert(loc = 14, column = 'REB_poss', value = (matchups2['REB']/matchups2['poss_per_game']))
matchups2.insert(loc = 15, column = 'OREB_poss', value = (matchups2['OREB']/matchups2['poss_per_game']))
matchups2.insert(loc = 16, column = 'DREB_poss', value = (matchups2['DREB']/matchups2['poss_per_game']))
matchups2.insert(loc = 17, column = 'AST_poss', value = (matchups2['AST']/matchups2['poss_per_game']))
matchups2.insert(loc = 18, column = 'TOV_poss', value = (matchups2['TOV']/matchups2['poss_per_game']))
matchups2.insert(loc = 19, column = 'STL_poss', value = (matchups2['STL']/matchups2['poss_per_game']))
matchups2.insert(loc = 20, column = 'BLK_poss', value = (matchups2['BLK']/matchups2['poss_per_game']))
matchups2.insert(loc = 21, column = 'FTM_poss', value = (matchups2['FTM']/matchups2['poss_per_game']))
matchups2.insert(loc = 22, column = 'FTA_poss', value = (matchups2['FTA']/matchups2['poss_per_game']))
matchups2.insert(loc = 21, column = 'FG3M_poss', value = (matchups2['FG3M']/matchups2['poss_per_game']))
matchups2.insert(loc = 22, column = 'FG3A_poss', value = (matchups2['FG3A']/matchups2['poss_per_game']))
matchups2.insert(loc = 21, column = 'FG2M_poss', value = (matchups2['FG2M']/matchups2['poss_per_game']))
matchups2.insert(loc = 22, column = 'FG2A_poss', value = (matchups2['FG2A']/matchups2['poss_per_game']))

#Drop matchups with zero possessions
matchups2 = matchups2[matchups2['PARTIAL_POSS'] != 0].copy()

Draftkings and Fanduel have slightly different formulae, so we're going to create separate columns for each. 

In [10]:
#Calculate fantasy points for each individual matchups
#DraftKings
matchups2.insert(loc = 1, column = 'MATCHUP_draftkings', value = (
        (matchups2.MATCHUP_FG3M*3.5)+(matchups2.MATCHUP_FG2M*2)+(matchups2.MATCHUP_FTM)+
        ((matchups2.REB_poss*matchups2.PARTIAL_POSS)*1.25)+
        ((matchups2.BLK_poss*matchups2.PARTIAL_POSS)*2)+
        ((matchups2.STL_poss*matchups2.PARTIAL_POSS)*2)+
        (matchups2.MATCHUP_TOV*(-0.5))+(matchups2.MATCHUP_AST*1.5)
        )
)
#FanDuel
matchups2.insert(loc = 1, column = 'MATCHUP_fanduel', value = (
        (matchups2.MATCHUP_FG3M*3)+(matchups2.MATCHUP_FG2M*2)+(matchups2.MATCHUP_FTM)+
        ((matchups2.REB_poss*matchups2.PARTIAL_POSS)*1.2)+
        ((matchups2.BLK_poss*matchups2.PARTIAL_POSS)*2)+
        ((matchups2.STL_poss*matchups2.PARTIAL_POSS)*2)+
        (matchups2.MATCHUP_TOV*(-1))+(matchups2.MATCHUP_AST*1.5)
        )
)

Now we will use the 'MATCHUP_' values and 'PARTIAL_POSS' to find per-possession values for each offesnive player/defensvie player matchup

In [11]:
#Calculate per-possession values for each matchup
matchups2.insert(loc = 10, column = 'MATCHUP_fanduel_poss', value = (matchups2['MATCHUP_fanduel']/matchups2['PARTIAL_POSS']))        
matchups2.insert(loc = 10, column = 'MATCHUP_draftkings_poss', value = (matchups2['MATCHUP_draftkings']/matchups2['PARTIAL_POSS']))
matchups2.insert(loc = 11, column = 'MATCHUP_PTS_poss', value = (matchups2['PLAYER_PTS']/matchups2['PARTIAL_POSS']))
matchups2.insert(loc = 12, column = 'MATCHUP_FGM_poss', value = (matchups2['MATCHUP_FGM']/matchups2['PARTIAL_POSS']))
matchups2.insert(loc = 13, column = 'MATCHUP_FGA_poss', value = (matchups2['MATCHUP_FGA']/matchups2['PARTIAL_POSS']))
matchups2.insert(loc = 17, column = 'MATCHUP_AST_poss', value = (matchups2['MATCHUP_AST']/matchups2['PARTIAL_POSS']))
matchups2.insert(loc = 18, column = 'MATCHUP_TOV_poss', value = (matchups2['MATCHUP_TOV']/matchups2['PARTIAL_POSS']))
matchups2.insert(loc = 20, column = 'MATCHUP_BLK_poss', value = (matchups2['MATCHUP_BLK']/matchups2['PARTIAL_POSS']))
matchups2.insert(loc = 21, column = 'MATCHUP_FTM_poss', value = (matchups2['MATCHUP_FTM']/matchups2['PARTIAL_POSS']))
matchups2.insert(loc = 22, column = 'MATCHUP_FTA_poss', value = (matchups2['MATCHUP_FTA']/matchups2['PARTIAL_POSS']))
matchups2.insert(loc = 21, column = 'MATCHUP_FG3M_poss', value = (matchups2['MATCHUP_FG3M']/matchups2['PARTIAL_POSS']))
matchups2.insert(loc = 22, column = 'MATCHUP_FG3A_poss', value = (matchups2['MATCHUP_FG3A']/matchups2['PARTIAL_POSS']))
matchups2.insert(loc = 21, column = 'MATCHUP_FG2M_poss', value = (matchups2['MATCHUP_FG2M']/matchups2['PARTIAL_POSS']))
matchups2.insert(loc = 22, column = 'MATCHUP_FG2A_poss', value = (matchups2['MATCHUP_FG2A']/matchups2['PARTIAL_POSS']))

The NBA's increase in precision has produced unreliable data where a defender will be listed as coering a cerain offensive player for 2.83 possessions and the offensive player scoring 8 points during those possession. The following block uses some logic to estimate a more accurate 'PARTIAL_POSS' value for these cases. 

In [12]:
#Step 1
#print(len(matchups2[matchups2['PARTIAL_POSS']%2 != 0]))
print(len(matchups2[matchups2['MATCHUP_draftkings_poss']>2]))
print(len(matchups2[matchups2['MATCHUP_draftkings_poss']>3]))
print(len(matchups2[matchups2['PARTIAL_POSS']<2]))

####Where PARTIAL_POSS goes to 1 decimal place
#Repairing PARTIAL_POSS where value goes to 1 decimal place and is even
matchups2['PARTIAL_POSS'].update(np.where(
(        
        (matchups2['PARTIAL_POSS']%2 != 0) & (matchups2['PARTIAL_POSS']%3 != 0) 
        & ((matchups2['PARTIAL_POSS']*10)%2 == 0)
        & (matchups2['PARTIAL_POSS']<5)
),
#For cases where condition is true
        round((matchups2['PARTIAL_POSS']%2)/2)
        + matchups2['PARTIAL_POSS']
,
#For cases where condition is false, and PARTIAL_POSS is an integer, odd, or goes to 2 decimal places
                                     matchups2['PARTIAL_POSS']
                                    )
                                    )

#Repairing PARTIAL_POSS where value goes to 1 decimal place and is odd
matchups2['PARTIAL_POSS'].update(np.where(
(
    (        
        (matchups2['PARTIAL_POSS']%2 != 0) & (matchups2['PARTIAL_POSS']%3 != 0) 
        & ((matchups2['PARTIAL_POSS']*10)%2 != 0) & (matchups2['PARTIAL_POSS']<5)
    )
    #Second group of conditions
    & (
        ((matchups2['PARTIAL_POSS']*10)%3 == 0) 
        | ((matchups2['PARTIAL_POSS']*10)%5 == 0) 
        | ((matchups2['PARTIAL_POSS']*10)%7 == 0)
    )
),
#For cases where condition is true
        round((matchups2['PARTIAL_POSS']%2)/2)
        + matchups2['PARTIAL_POSS']
,
#For cases where condition is false, and either 1) PARTIAL_POSS is integer or 2) PARTIAL_POSS has goes to 2 decimal places
                                     matchups2['PARTIAL_POSS']
                                    )
                                    )


#Repairing PARTIAL_POSS where value goes to 2 decimal places and is even
matchups2['PARTIAL_POSS'].update(np.where(
(        
        (matchups2['PARTIAL_POSS']%2 != 0) & (matchups2['PARTIAL_POSS']%3 != 0) 
        & ((matchups2['PARTIAL_POSS']*100)%2 == 0)
        & (matchups2['PARTIAL_POSS']<5)
),
#For cases where condition is true
        (((matchups2['PARTIAL_POSS']*10)%2)/2)
        + matchups2['PARTIAL_POSS']
,
#For cases where condition is false, and either 1) PARTIAL_POSS is integer or 2) PARTIAL_POSS is odd
                                     matchups2['PARTIAL_POSS']
                                    )
                                    )


matchups2['MATCHUP_fanduel_poss'].update(matchups2['MATCHUP_fanduel'].astype(float)/matchups2['PARTIAL_POSS'].astype(float))
matchups2['MATCHUP_draftkings_poss'].update(matchups2['MATCHUP_draftkings'].astype(float)/matchups2['PARTIAL_POSS'].astype(float))


### Where PARTIAL_POSS goes to 2 decimal places
#Repairing PARTIAL_POSS where value goes to 2 decimal places and is odd, but divisin;e by a number less than 20
matchups2['PARTIAL_POSS'].update(np.where(
(
    (        
        (matchups2['PARTIAL_POSS']%2 != 0) & (matchups2['PARTIAL_POSS']%3 != 0) 
        & ((matchups2['PARTIAL_POSS']*100)%2 != 0) & (matchups2['PARTIAL_POSS']<5)
    )
    #Second group of conditions
    & (
        ((matchups2['PARTIAL_POSS']*100)%3 == 0) 
        | ((matchups2['PARTIAL_POSS']*100)%5 == 0) 
        | ((matchups2['PARTIAL_POSS']*100)%7 == 0)
        | ((matchups2['PARTIAL_POSS']*100)%11 == 0)
        | ((matchups2['PARTIAL_POSS']*100)%13 == 0)
        | ((matchups2['PARTIAL_POSS']*100)%17 == 0)
        | ((matchups2['PARTIAL_POSS']*100)%19 == 0)
    )
),
#For cases where condition is true
        ((round(((matchups2['PARTIAL_POSS']*10)%2)/2))
        + (round(matchups2['PARTIAL_POSS'])))
,
#For cases where condition is false, and either 1) PARTIAL_POSS is integer or 2) PARTIAL_POSS has goes to 2 decimal places
                                     matchups2['PARTIAL_POSS']
                                    )
                                    )


matchups2['MATCHUP_fanduel_poss'].update(matchups2['MATCHUP_fanduel'].astype(float)/matchups2['PARTIAL_POSS'].astype(float))
matchups2['MATCHUP_draftkings_poss'].update(matchups2['MATCHUP_draftkings'].astype(float)/matchups2['PARTIAL_POSS'].astype(float))

#Repairing PARTIAL_POSS where MATCHUP_PTS_poss > 2 and PARTIAL_POSS < 5
matchups2['PARTIAL_POSS'].update(np.where(   
        (matchups2['MATCHUP_PTS_poss']>2) & (matchups2['PARTIAL_POSS']<5)
,
#For cases where condition is true
        ((matchups2['PARTIAL_POSS']%2)/2)
        + matchups2['PARTIAL_POSS']
,
#For cases where condition is false, and either 1) PARTIAL_POSS is integer or 2) PARTIAL_POSS has goes to 2 decimal places
                                     matchups2['PARTIAL_POSS']
                                    )
                                    )



matchups2['MATCHUP_fanduel_poss'].update(matchups2['MATCHUP_fanduel'].astype(float)/matchups2['PARTIAL_POSS'].astype(float))
matchups2['MATCHUP_draftkings_poss'].update(matchups2['MATCHUP_draftkings'].astype(float)/matchups2['PARTIAL_POSS'].astype(float))

#print(len(matchups2[matchups2['PARTIAL_POSS']%2 != 0]))
print(len(matchups2[matchups2['MATCHUP_draftkings_poss']>2]))
print(len(matchups2[matchups2['MATCHUP_draftkings_poss']>3]))
print(len(matchups2[matchups2['PARTIAL_POSS']<2]))

54587
20084
358349
38939
3353
319337


In [13]:
#matchups2['MATCHUP_fanduel_poss'].update(matchups2['MATCHUP_fanduel'].astype(float)/matchups2['PARTIAL_POSS'])
#matchups2['MATCHUP_draftkings_poss'].update(matchups2['MATCHUP_draftkings'].astype(float)/matchups2['PARTIAL_POSS'])

In [14]:
#Hard-capping extreme results from small samples
matchups2['MATCHUP_PTS_poss'].update(np.where(matchups2['MATCHUP_PTS_poss']>3, 3, matchups2['MATCHUP_PTS_poss']))
matchups2['MATCHUP_fanduel_poss'].update(np.where(matchups2['MATCHUP_fanduel_poss']>3, 3, matchups2['MATCHUP_fanduel_poss']))
matchups2['MATCHUP_draftkings_poss'].update(np.where(matchups2['MATCHUP_draftkings_poss']>3, 3, matchups2['MATCHUP_draftkings_poss']))

No we compare the players performance against the given defender with his average performance to determine the differential caused by that defender. 

In [15]:
#Calculate differentials caused by defender
matchups2.insert(loc = 11, column = 'MATCHUP_draftkings_diff', 
                 value = (matchups2['MATCHUP_draftkings_poss']-matchups2['draftkings_poss']))
matchups2.insert(loc = 12, column = 'MATCHUP_fanduel_diff', 
                 value = (matchups2['MATCHUP_fanduel_poss']-matchups2['fanduel_poss']))
matchups2.insert(loc = 11, column = 'MATCHUP_PTS_diff', value = (matchups2['MATCHUP_PTS_poss']-matchups2['PTS_poss']))
matchups2.insert(loc = 12, column = 'MATCHUP_FGM_diff', value = (matchups2['MATCHUP_FGM_poss']-matchups2['FGM_poss']))
matchups2.insert(loc = 13, column = 'MATCHUP_FGA_diff', value = (matchups2['MATCHUP_FGA_poss']-matchups2['FGA_poss']))
matchups2.insert(loc = 17, column = 'MATCHUP_AST_diff', value = (matchups2['MATCHUP_AST_poss']-matchups2['AST_poss']))
matchups2.insert(loc = 18, column = 'MATCHUP_TOV_diff', value = (matchups2['MATCHUP_TOV_poss']-matchups2['TOV_poss']))
matchups2.insert(loc = 20, column = 'MATCHUP_BLK_diff', value = (matchups2['MATCHUP_BLK_poss']-matchups2['BLK_poss']))
matchups2.insert(loc = 21, column = 'MATCHUP_FTM_diff', value = (matchups2['MATCHUP_FTM_poss']-matchups2['FTM_poss']))
matchups2.insert(loc = 22, column = 'MATCHUP_FTA_diff', value = (matchups2['MATCHUP_FTA_poss']-matchups2['FTA_poss']))
matchups2.insert(loc = 21, column = 'MATCHUP_FG3M_diff', value = (matchups2['MATCHUP_FG3M_poss']-matchups2['FG3M_poss']))
matchups2.insert(loc = 22, column = 'MATCHUP_FG3A_diff', value = (matchups2['MATCHUP_FG3A_poss']-matchups2['FG3A_poss']))
matchups2.insert(loc = 21, column = 'MATCHUP_FG2M_diff', value = (matchups2['MATCHUP_FG2M_poss']-matchups2['FG2M_poss']))
matchups2.insert(loc = 22, column = 'MATCHUP_FG2A_diff', value = (matchups2['MATCHUP_FG2A_poss']-matchups2['FG2A_poss']))

# Scrape data for today's games

###   Scrape schedule

The function scrape_schedule pulls the schedule for a given day from the NBA schedule endpoint and indicates which team is home and which is away.

In [16]:
def scrape_schedule ():
    headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36',
    'Referer': 'https://www.nba.com/',
    }
    url = 'https://cdn.nba.com/static/json/staticData/scheduleLeagueV2.json'
    r = requests.get(url, headers=headers, allow_redirects = False).json()
    date_list = r['leagueSchedule']['gameDates']
    game_frames = []
    for date in date_list:
        for game in date['games']:
            day = {key:value for key,value in game.items() if key == 'gameDateEst'}
            game_dict = {key:value for key,value in game.items() if key == 'gameId'}
            home = {key:value for key,value in game.items() if key == 'homeTeam'}
            away = {key:value for key,value in game.items() if key == 'awayTeam'}
            for team in home.values():
                home_team = team['teamId']
            for team in away.values():
                away_team = team['teamId']
            game_dict['Home'] = home_team
            game_dict['Away'] = away_team
            game_dict['Date'] = day.values()
            df = pd.DataFrame.from_dict(game_dict, orient = 'index').swapaxes("index","columns")
            game_frames.append(df)
    schedule = pd.concat(game_frames).set_index('gameId')
    schedule['Date'] = schedule['Date'].astype(str).copy()
    schedule['Date'] = schedule['Date'].str[14:24].copy()
    return schedule

# Do not delete

We are here reshaping the output of scrape_schedule in order to essentially perform a self-join so that home team and away team are both associated iwth the same game id.

In [17]:
#Pull schedule and filter for today's date
schedule = scrape_schedule()
schedule['GAME_ID'] = schedule.index
schedule.insert(loc = 4, column = 'Season',value = ("20"+schedule['GAME_ID'].str[3:5]+"-"+((schedule['GAME_ID'].str[3:5].astype(int))+1).astype(str)))

#The following line only temporarily omitted and MUST be reinstated for production
#today = schedule[schedule['Date'] == todays_date]

#Temporary replacement for above line which must be REMOVED for production
today = schedule

#Format from home team's POV
home = today.rename(columns = {"Home":"Team", "Away":"Opponent"})
home.insert(loc = 5,column = 'Location',value = 'H')

#Format from visitor's POV
away = today.rename(columns = {"Home":"Opponent", "Away":"Team"})
away = away[['Team','Opponent','Date','GAME_ID','Season']]
away.insert(loc = 5,column = 'Location',value = 'A')

#Combine home and away
sched = [home,away]
today2 = pd.concat(sched)

###  Scrape rosters

In [18]:
#Scrape commonteamrosters
def scrape_commonteamrosters():
    headers = {
'Host': 'stats.nba.com',
'Connection': 'keep-alive',
'Accept': 'application/json, text/plain, */*',
'x-nba-stats-token': 'true',
'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36',
'x-nba-stats-origin': 'stats',
'Referer': 'https://www.nba.com/',
'Accept-Encoding': 'gzip, deflate, br',
'Accept-Language': 'en-US,en;q=0.9'}
    
    rosters = []
    for team in range(1610612737,1610612768):
        time.sleep(np.random.randint(0,5 + 1))
        url = f"https://stats.nba.com/stats/commonteamroster?LeagueID=00&Season=2019-20&TeamID={team}"
        r = requests.get(url, headers=headers).json()
        df = pd.DataFrame(r['resultSets'][0]['rowSet'], columns = r['resultSets'][0]['headers'])
        rosters.append(df)

    return rosters

### #  Matchups3

After scraping the rosters, we join them to the schedule so that we have only the roster for teams playing today. With the rosters in hand, we need to join with matchups2_slim (thinner version with unecessary columns dropped) 

In [19]:
#Get "offensive" rosters
roster_list = scrape_commonteamrosters()
rosters = pd.concat(roster_list)
todays_rosters = pd.merge(left = today2, right = rosters, how = 'inner', left_on = ['Team'], right_on = ['TeamID'])

#Review whether to use trimmed form or thick form here
matchups2_slim = matchups2[['PLAYER_ID','PLAYER_NAME','Season','fanduel','draftkings','TEAM_ID','TEAM_ABBREVIATION',
                    'fanduel_poss','draftkings_poss','total_possessions','poss_per_game','GP_x',
                   'PTS_poss','FGM_poss','FGA_poss','FG3M_poss','FG3A_poss','FG2M_poss','FG2A_poss','FTM_poss',
                   'FTA_poss','REB_poss','AST_poss','TOV_poss','STL_poss','BLK_poss']].drop_duplicates()
matchups3 = pd.merge(left = matchups2_slim, right = todays_rosters, how = 'inner', left_on = ['PLAYER_ID'], right_on = ['PLAYER_ID'])
matchups3 = matchups3[(matchups3['Season_x'] == this_year) | (matchups3['Season_x'] == last_year)].copy()
matchups3 = matchups3.drop(['SEASON','LeagueID','PLAYER','PLAYER_SLUG','BIRTH_DATE','EXP','SCHOOL','NUM'], axis = 1)

### # Matchups4, matchups5

Here we are merging the roster for the "defensive team" with the rosters for the "offensive team"

In [22]:
#print(len(opponent))
print(len(matchups3))
#opponent.head()

35881


In [23]:
defenders = matchups2[['SEASON_ID','OFF_PLAYER_ID','OFF_PLAYER_NAME','DEF_PLAYER_ID','DEF_PLAYER_NAME','PARTIAL_POSS',
                       'MATCHUP_fanduel_poss','MATCHUP_draftkings_poss','MATCHUP_PTS_poss','MATCHUP_FGM_poss','MATCHUP_FGA_poss',
                      'MATCHUP_FG3M_poss','MATCHUP_FG3A_poss','MATCHUP_FG2M_poss','MATCHUP_FG2A_poss','MATCHUP_FTM_poss',
                      'MATCHUP_FTA_poss','MATCHUP_AST_poss','MATCHUP_TOV_poss','REB_poss','STL_poss','BLK_poss',
                       'MATCHUP_fanduel_diff','MATCHUP_draftkings_diff','MATCHUP_PTS_diff','MATCHUP_FGM_diff','MATCHUP_FGA_diff',
                      'MATCHUP_FG3M_diff','MATCHUP_FG3A_diff','MATCHUP_FG2M_diff','MATCHUP_FG2A_diff','MATCHUP_FTM_diff',
                      'MATCHUP_FTA_diff','MATCHUP_AST_diff','MATCHUP_TOV_diff']]

#Join schedule to opponent roster
opponent = pd.merge(left = today2, right = rosters, how = 'inner', left_on = ['Opponent'], right_on = ['TeamID'])

#Join matchups3 to opponent
matchups4 = pd.merge (left = matchups3, right = opponent, how = 'outer', left_on = ['TEAM_ID'], right_on = ['Team'])



#Drop unnecessary columns
matchups4 = matchups4.drop(['Team_x','Opponent_x','Date_x','GAME_ID_x','Season_y','POSITION_x','HEIGHT_x','WEIGHT_x','AGE_x',
                           'PLAYER_SLUG','LeagueID','NUM','BIRTH_DATE','EXP','SCHOOL','TeamID_x','Location_y'], axis = 1)

#Join matchups4 to defenders
matchups5 = pd.merge(left = matchups4, right = defenders, 
                     how = 'inner', left_on = ['PLAYER_ID_x','PLAYER_ID_y'], right_on = ['OFF_PLAYER_ID','DEF_PLAYER_ID'])

MemoryError: Unable to allocate 4.04 GiB for an array with shape (22, 24671630) and data type float64

Now we need to aggregate across every instacne of a pairing. For most offensvie player/defensive payer matchups, the two have faced each other in mroe than one game. So we no need to aggregate across games. 

In [ ]:
matchups5.insert(loc = 35, column = 'PARTIAL_POSS2', value = (matchups5.groupby(['OFF_PLAYER_ID','DEF_PLAYER_ID'])['PARTIAL_POSS'].transform('sum')))
matchups5.insert(loc = 36, column = 'MATCHUP_fanduel_diff2', value = (matchups5.groupby(['OFF_PLAYER_ID','DEF_PLAYER_ID'])['MATCHUP_fanduel_diff'].transform('sum')))
matchups5.insert(loc = 37, column = 'MATCHUP_draftkings_diff2', value = (matchups5.groupby(['OFF_PLAYER_ID','DEF_PLAYER_ID'])['MATCHUP_draftkings_diff'].transform('sum')))
matchups5.insert(loc = 38, column = 'MATCHUP_PTS_diff2', value = (matchups5.groupby(['OFF_PLAYER_ID','DEF_PLAYER_ID'])['MATCHUP_PTS_diff'].transform('sum')))
matchups5.insert(loc = 39, column = 'MATCHUP_FGM_diff2', value = (matchups5.groupby(['OFF_PLAYER_ID','DEF_PLAYER_ID'])['MATCHUP_FGM_diff'].transform('sum')))
matchups5.insert(loc = 40, column = 'MATCHUP_FGA_diff2', value = (matchups5.groupby(['OFF_PLAYER_ID','DEF_PLAYER_ID'])['MATCHUP_FGA_diff'].transform('sum')))
matchups5.insert(loc = 41, column = 'MATCHUP_FG2M_diff2', value = (matchups5.groupby(['OFF_PLAYER_ID','DEF_PLAYER_ID'])['MATCHUP_FG2M_diff'].transform('sum')))
matchups5.insert(loc = 42, column = 'MATCHUP_FG2A_diff2', value = (matchups5.groupby(['OFF_PLAYER_ID','DEF_PLAYER_ID'])['MATCHUP_FG2A_diff'].transform('sum')))
matchups5.insert(loc = 43, column = 'MATCHUP_FG3M_diff2', value = (matchups5.groupby(['OFF_PLAYER_ID','DEF_PLAYER_ID'])['MATCHUP_FG3M_diff'].transform('sum')))
matchups5.insert(loc = 44, column = 'MATCHUP_FG3A_diff2', value = (matchups5.groupby(['OFF_PLAYER_ID','DEF_PLAYER_ID'])['MATCHUP_FG3A_diff'].transform('sum')))
matchups5.insert(loc = 45, column = 'MATCHUP_FTM_diff2', value = (matchups5.groupby(['OFF_PLAYER_ID','DEF_PLAYER_ID'])['MATCHUP_FTM_diff'].transform('sum')))
matchups5.insert(loc = 46, column = 'MATCHUP_FTA_diff2', value = (matchups5.groupby(['OFF_PLAYER_ID','DEF_PLAYER_ID'])['MATCHUP_FTA_diff'].transform('sum')))
matchups5.insert(loc = 47, column = 'MATCHUP_AST_diff2', value = (matchups5.groupby(['OFF_PLAYER_ID','DEF_PLAYER_ID'])['MATCHUP_AST_diff'].transform('sum')))
matchups5.insert(loc = 48, column = 'MATCHUP_TOV_diff2', value = (matchups5.groupby(['OFF_PLAYER_ID','DEF_PLAYER_ID'])['MATCHUP_TOV_diff'].transform('sum')))

Next we update the "per possession" values for each matchup with the previosuly calculated values that aggregate across games.

In [ ]:
matchups5['PARTIAL_POSS'].update(matchups5['PARTIAL_POSS2'])
matchups5['MATCHUP_fanduel_diff'].update(matchups5['MATCHUP_fanduel_diff2'])
matchups5['MATCHUP_draftkings_diff'].update(matchups5['MATCHUP_draftkings_diff2'])
matchups5['MATCHUP_PTS_diff'].update(matchups5['MATCHUP_PTS_diff2'])
matchups5['MATCHUP_FGM_diff'].update(matchups5['MATCHUP_FGM_diff2'])
matchups5['MATCHUP_FGA_diff'].update(matchups5['MATCHUP_FGA_diff2'])
matchups5['MATCHUP_FG2M_diff'].update(matchups5['MATCHUP_FG2M_diff2'])
matchups5['MATCHUP_FG2A_diff'].update(matchups5['MATCHUP_FG2A_diff2'])
matchups5['MATCHUP_FG3M_diff'].update(matchups5['MATCHUP_FG3M_diff2'])
matchups5['MATCHUP_FG3A_diff'].update(matchups5['MATCHUP_FG3A_diff2'])
matchups5['MATCHUP_FTM_diff'].update(matchups5['MATCHUP_FTM_diff2'])
matchups5['MATCHUP_FTA_diff'].update(matchups5['MATCHUP_FTA_diff2'])
matchups5['MATCHUP_AST_diff'].update(matchups5['MATCHUP_AST_diff2'])
matchups5['MATCHUP_TOV_diff'].update(matchups5['MATCHUP_TOV_diff2'])

### Matchups6

The purpose of matchups6 is to aggregate the box score *values* for each matchup. Previously, we have only aggregated the *differentials*, but not the simple number of FG3M, AST, TOV, etc.

In [ ]:
matchup_raw = matchups2[['OFF_PLAYER_ID','DEF_PLAYER_ID','PLAYER_PTS','MATCHUP_FGM','MATCHUP_FGA','MATCHUP_FG2M',
                         'MATCHUP_FG2A','MATCHUP_FG3M','MATCHUP_FG3A','MATCHUP_FTM','MATCHUP_FTA','MATCHUP_AST','MATCHUP_TOV',
                        'MATCHUP_fanduel','MATCHUP_draftkings']]
matchup_raw.insert(loc = 1, column = 'OFFENSIVE_PLAYER', value = matchup_raw['OFF_PLAYER_ID'])
matchup_raw.insert(loc = 1, column = 'DEFENSIVE_PLAYER', value = matchup_raw['DEF_PLAYER_ID'])
matchup_raw = matchup_raw.drop(['OFF_PLAYER_ID','DEF_PLAYER_ID'], axis = 1)

#matchups 5 left join matchup_raw
matchups6 = pd.merge(left = matchups5, right = matchup_raw, how = 'left', left_on = ['OFF_PLAYER_ID','DEF_PLAYER_ID',], right_on = ['OFFENSIVE_PLAYER','DEFENSIVE_PLAYER'])

In [ ]:
matchups6.insert(loc = 36, column = 'MATCHUP_fanduel2', value = (matchups6.groupby(['OFF_PLAYER_ID','DEF_PLAYER_ID'])['MATCHUP_fanduel'].transform('sum')))
matchups6.insert(loc = 37, column = 'MATCHUP_draftkings2', value = (matchups6.groupby(['OFF_PLAYER_ID','DEF_PLAYER_ID'])['MATCHUP_draftkings'].transform('sum')))
matchups6.insert(loc = 38, column = 'MATCHUP_PTS2', value = (matchups6.groupby(['OFF_PLAYER_ID','DEF_PLAYER_ID'])['PLAYER_PTS'].transform('sum')))
matchups6.insert(loc = 39, column = 'MATCHUP_FGM2', value = (matchups6.groupby(['OFF_PLAYER_ID','DEF_PLAYER_ID'])['MATCHUP_FGM'].transform('sum')))
matchups6.insert(loc = 40, column = 'MATCHUP_FGA2', value = (matchups6.groupby(['OFF_PLAYER_ID','DEF_PLAYER_ID'])['MATCHUP_FGA'].transform('sum')))
matchups6.insert(loc = 41, column = 'MATCHUP_FG2M2', value = (matchups6.groupby(['OFF_PLAYER_ID','DEF_PLAYER_ID'])['MATCHUP_FG2M'].transform('sum')))
matchups6.insert(loc = 42, column = 'MATCHUP_FG2A2', value = (matchups6.groupby(['OFF_PLAYER_ID','DEF_PLAYER_ID'])['MATCHUP_FG2A'].transform('sum')))
matchups6.insert(loc = 43, column = 'MATCHUP_FG3M2', value = (matchups6.groupby(['OFF_PLAYER_ID','DEF_PLAYER_ID'])['MATCHUP_FG3M'].transform('sum')))
matchups6.insert(loc = 44, column = 'MATCHUP_FG3A2', value = (matchups6.groupby(['OFF_PLAYER_ID','DEF_PLAYER_ID'])['MATCHUP_FG3A'].transform('sum')))
matchups6.insert(loc = 45, column = 'MATCHUP_FTM2', value = (matchups6.groupby(['OFF_PLAYER_ID','DEF_PLAYER_ID'])['MATCHUP_FTM'].transform('sum')))
matchups6.insert(loc = 46, column = 'MATCHUP_FTA2', value = (matchups6.groupby(['OFF_PLAYER_ID','DEF_PLAYER_ID'])['MATCHUP_FTA'].transform('sum')))
matchups6.insert(loc = 47, column = 'MATCHUP_AST2', value = (matchups6.groupby(['OFF_PLAYER_ID','DEF_PLAYER_ID'])['MATCHUP_AST'].transform('sum')))
matchups6.insert(loc = 48, column = 'MATCHUP_TOV2', value = (matchups6.groupby(['OFF_PLAYER_ID','DEF_PLAYER_ID'])['MATCHUP_TOV'].transform('sum')))

Update the original fields with the aggregtd values so that we can drop duplicates

In [ ]:
matchups6['MATCHUP_fanduel'].update(matchups6['MATCHUP_fanduel2'])
matchups6['MATCHUP_draftkings'].update(matchups6['MATCHUP_draftkings2'])
matchups6['MATCHUP_FGM'].update(matchups6['MATCHUP_FGM2'])
matchups6['MATCHUP_FGA'].update(matchups6['MATCHUP_FGA2'])
matchups6['MATCHUP_FG2M'].update(matchups6['MATCHUP_FG2M2'])
matchups6['MATCHUP_FG2A'].update(matchups6['MATCHUP_FG2A2'])
matchups6['MATCHUP_FG3M'].update(matchups6['MATCHUP_FG3M2'])
matchups6['MATCHUP_FG3A'].update(matchups6['MATCHUP_FG3A2'])
matchups6['MATCHUP_FTM'].update(matchups6['MATCHUP_FTM2'])
matchups6['MATCHUP_FTA'].update(matchups6['MATCHUP_FTA2'])
matchups6['MATCHUP_AST'].update(matchups6['MATCHUP_AST2'])
matchups6['MATCHUP_TOV'].update(matchups6['MATCHUP_TOV2'])


matchups6.insert(loc = 117, column = 'MATCHUP_PTS', value = matchups6['MATCHUP_PTS2'])

In [ ]:
#Update matchup per-possession values to include all possessions, regardless of season
matchups6['MATCHUP_fanduel_poss'].update(matchups6['MATCHUP_fanduel']/matchups6['PARTIAL_POSS'])
matchups6['MATCHUP_draftkings_poss'].update(matchups6['MATCHUP_draftkings']/matchups6['PARTIAL_POSS'])
matchups6['MATCHUP_PTS_poss'].update(matchups6['MATCHUP_PTS']/matchups6['PARTIAL_POSS'])
matchups6['MATCHUP_FGM_poss'].update(matchups6['MATCHUP_FGM']/matchups6['PARTIAL_POSS'])
matchups6['MATCHUP_FGA_poss'].update(matchups6['MATCHUP_FGA']/matchups6['PARTIAL_POSS'])
matchups6['MATCHUP_FG3M_poss'].update(matchups6['MATCHUP_FG3M']/matchups6['PARTIAL_POSS'])
matchups6['MATCHUP_FG3A_poss'].update(matchups6['MATCHUP_FG3A']/matchups6['PARTIAL_POSS'])
matchups6['MATCHUP_FG2M_poss'].update(matchups6['MATCHUP_FG2M']/matchups6['PARTIAL_POSS'])
matchups6['MATCHUP_FG2A_poss'].update(matchups6['MATCHUP_FG2A']/matchups6['PARTIAL_POSS'])
matchups6['MATCHUP_FTM_poss'].update(matchups6['MATCHUP_FTM']/matchups6['PARTIAL_POSS'])
matchups6['MATCHUP_FTA_poss'].update(matchups6['MATCHUP_FTA']/matchups6['PARTIAL_POSS'])
matchups6['MATCHUP_AST_poss'].update(matchups6['MATCHUP_AST']/matchups6['PARTIAL_POSS'])
matchups6['MATCHUP_TOV_poss'].update(matchups6['MATCHUP_TOV']/matchups6['PARTIAL_POSS'])


matchups6 = matchups6.drop(['PLAYER_PTS'], axis = 1).drop_duplicates()
matchups6 = matchups6[(matchups6['SEASON_ID'] == '2019-20') | (matchups6['SEASON_ID'] == '2018-19')]

In [ ]:
#Hard-capping unreliable per-possession values derived from PARTIAL_POSS < 1
matchups6.loc[matchups6['MATCHUP_fanduel_poss'] > 3, 'MATCHUP_fanduel_poss'] = 3
matchups6.loc[matchups6['MATCHUP_draftkings_poss'] > 3, 'MATCHUP_draftkings_poss'] = 3
matchups6.loc[matchups6['MATCHUP_PTS_poss'] > 3, 'MATCHUP_PTS_poss'] = 3

### # Matchups7

In [ ]:
matchups7 = matchups6.drop(['PARTIAL_POSS2','MATCHUP_fanduel2','MATCHUP_draftkings2','MATCHUP_PTS2','MATCHUP_FGM2',
                            'MATCHUP_FGA2','MATCHUP_FG2M2','MATCHUP_FG2A2','MATCHUP_FG3M2','MATCHUP_FG3A2','MATCHUP_FTM2',
                           'MATCHUP_FTA2','MATCHUP_AST2','MATCHUP_TOV2','MATCHUP_fanduel_diff2','MATCHUP_draftkings_diff2',
                           'MATCHUP_PTS_diff2','MATCHUP_FGM_diff2','MATCHUP_FGA_diff2','MATCHUP_FG2M_diff2',
                            'MATCHUP_FG2A_diff2','MATCHUP_FG3M_diff2','MATCHUP_FG3A_diff2','MATCHUP_FTM_diff2',
                           'MATCHUP_FTA_diff2','MATCHUP_AST_diff2','MATCHUP_TOV_diff2','OFFENSIVE_PLAYER',
                           'DEFENSIVE_PLAYER','REB_poss_y','STL_poss_y','BLK_poss_y'], axis = 1)

The dataframe game_counter is a sub-table used to determine how much to weight this year's data relative to last year's data.

In [ ]:
#Building separate df to derive weights
game_counter = matchups3[['PLAYER_ID','Season_x','GP_x','PTS_poss','FGM_poss','FGA_poss','FG2M_poss','FG2A_poss',
                         'FG3M_poss','FG3A_poss','FTM_poss','FTA_poss','AST_poss','REB_poss','TOV_poss','STL_poss',
                         'BLK_poss','fanduel_poss','draftkings_poss','PLAYER_NAME']].drop_duplicates()

game_counter.insert(loc = 3, column = 'weight', value = 
    (game_counter['GP_x'] / (game_counter.groupby(['PLAYER_ID'])['GP_x'].transform('sum'))))
game_counter = game_counter.rename(mapper = {"PLAYER_ID":"ID","Season_x":"SEASON_ID"}, axis = 1)
game_counter = game_counter.drop('GP_x', axis = 1)

In [ ]:
#Derive weighting for each stat in each season
game_counter['fd'] = game_counter['fanduel_poss']*game_counter['weight']
game_counter['dk'] = game_counter['draftkings_poss']*game_counter['weight']
game_counter['pts'] = game_counter['PTS_poss']*game_counter['weight']
game_counter['fgm'] = game_counter['FGM_poss']*game_counter['weight']
game_counter['fga'] = game_counter['FGA_poss']*game_counter['weight']
game_counter['fg2m'] = game_counter['FG2M_poss']*game_counter['weight']
game_counter['fg2a'] = game_counter['FG2A_poss']*game_counter['weight']
game_counter['fg3m'] = game_counter['FG3M_poss']*game_counter['weight']
game_counter['fg3a'] = game_counter['FG3A_poss']*game_counter['weight']
game_counter['ftm'] = game_counter['FTM_poss']*game_counter['weight']
game_counter['fta'] = game_counter['FTA_poss']*game_counter['weight']
game_counter['reb'] = game_counter['REB_poss']*game_counter['weight']
game_counter['ast'] = game_counter['AST_poss']*game_counter['weight']
game_counter['tov'] = game_counter['TOV_poss']*game_counter['weight']
game_counter['stl'] = game_counter['STL_poss']*game_counter['weight']
game_counter['blk'] = game_counter['BLK_poss']*game_counter['weight']

In [ ]:
#Calculating weighted averages
game_counter['fanduel_poss'].update(game_counter.groupby(['ID'])['fd'].transform('sum'))
game_counter['draftkings_poss'].update(game_counter.groupby(['ID'])['dk'].transform('sum'))
game_counter['PTS_poss'].update(game_counter.groupby(['ID'])['pts'].transform('sum'))
game_counter['FGM_poss'].update(game_counter.groupby(['ID'])['fgm'].transform('sum'))
game_counter['FGA_poss'].update(game_counter.groupby(['ID'])['fga'].transform('sum'))
game_counter['FG2M_poss'].update(game_counter.groupby(['ID'])['fg2m'].transform('sum'))
game_counter['FG2A_poss'].update(game_counter.groupby(['ID'])['fg2a'].transform('sum'))
game_counter['FG3M_poss'].update(game_counter.groupby(['ID'])['fg3m'].transform('sum'))
game_counter['FG3A_poss'].update(game_counter.groupby(['ID'])['fg3a'].transform('sum'))
game_counter['FTM_poss'].update(game_counter.groupby(['ID'])['ftm'].transform('sum'))
game_counter['FTA_poss'].update(game_counter.groupby(['ID'])['fta'].transform('sum'))
game_counter['AST_poss'].update(game_counter.groupby(['ID'])['ast'].transform('sum'))
game_counter['TOV_poss'].update(game_counter.groupby(['ID'])['tov'].transform('sum'))
game_counter['REB_poss'].update(game_counter.groupby(['ID'])['reb'].transform('sum'))
game_counter['STL_poss'].update(game_counter.groupby(['ID'])['stl'].transform('sum'))
game_counter['BLK_poss'].update(game_counter.groupby(['ID'])['blk'].transform('sum'))

#Renaming
game_counter = game_counter.rename(columns = {'fanduel_poss':'baseline_fanduel','draftkings_poss':'baseline_draftkings',
        'PTS_poss':'baseline_pts','FGM_poss':'baseline_fgm','FGA_poss':'baseline_fga','FG2M_poss':'baseline_fg2m',
        'FG3M_poss':'baseline_fg3m','FG3A_poss':'baseline_fg3a','FTM_poss':'baseline_ftm','FTA_poss':'baseline_fta',
        'AST_poss':'baseline_ast','TOV_poss':'baseline_tov','REB_poss_x':'baseline_reb','STL_poss_x':'baseline_stl',
        'BLK_poss_x':'baseline_blk','FG2M_poss':'baseline_fg2m'})


#Dropping unecessary columns
game_counter = game_counter.drop(['SEASON_ID','weight','fd','dk','pts','fgm','fga','fg2m','fg2a','fg3m','fg3a','ftm','fta','reb',
                                 'ast','tov','stl','blk'], axis = 1)
game_counter = game_counter.drop_duplicates()

In [ ]:
#Take in only row for current season in matchups7
matchups7 = matchups7[matchups7['Season_x'] == this_year]
#matchups7 = matchups7.drop(columns = ['SEASON_ID']).drop_duplicates()

### # Matchups8

Now that we've weighted this year vs last year, we know how much we should realistically expect each player to produce.

In [ ]:
#Joining weights to matchups7
matchups8 = pd.merge(left = matchups7, right = game_counter,how = 'inner', left_on = ['PLAYER_ID_x'], 
                     right_on = ['ID'])

matchups8.insert(loc = 101, column = 'total_defensive_impact_fanduel',
                 value = matchups8.groupby(['PLAYER_ID_x','Opponent_y'])['MATCHUP_fanduel_diff'].transform('sum'))
matchups8.insert(loc = 101, column = 'total_defensive_impact_draftkings',
                 value = matchups8.groupby(['OFF_PLAYER_ID','Opponent_y'])['MATCHUP_draftkings_diff'].transform('sum'))
matchups8.insert(loc = 101, column = 'total_defensive_impact_pts',
                 value = matchups8.groupby(['OFF_PLAYER_ID','Opponent_y'])['MATCHUP_PTS_diff'].transform('sum'))

In [ ]:
#Calculating forecasted effect for individual defender
matchups8.insert(loc = 101, column = 'total_defensive_poss',
                 value = matchups8.groupby(['OFF_PLAYER_ID','Opponent_y'])['PARTIAL_POSS'].transform('sum'))
matchups8.insert(loc = 101, column = 'individual_defender_impact_fanduel',
                 value = (matchups8['poss_per_game']*
                          (matchups8['MATCHUP_fanduel_diff']/matchups8['PARTIAL_POSS'])*
                          (matchups8['PARTIAL_POSS']/matchups8['total_defensive_poss'])
                         )
                )
matchups8.insert(loc = 101, column = 'individual_defender_impact_draftkings',
                 value = (matchups8['poss_per_game']*
                          (matchups8['MATCHUP_draftkings_diff']/matchups8['PARTIAL_POSS'])*
                          (matchups8['PARTIAL_POSS']/matchups8['total_defensive_poss'])
                         )
                )
matchups8.insert(loc = 101, column = 'individual_defender_impact_pts',
                 value = (matchups8['poss_per_game']*
                          (matchups8['MATCHUP_PTS_diff']/matchups8['PARTIAL_POSS'])*
                          (matchups8['PARTIAL_POSS']/matchups8['total_defensive_poss'])
                         )
                )

In [ ]:
#Aggregating effect across offensive player
matchups8.insert(loc = 100, column = 'impact_of_defenders_fanduel', 
                 value = matchups8.groupby(['PLAYER_ID_x'])['individual_defender_impact_fanduel'].transform('sum'))
matchups8.insert(loc = 100, column = 'impact_of_defenders_draftkings', 
                 value = matchups8.groupby(['PLAYER_ID_x'])['individual_defender_impact_draftkings'].transform('sum'))
matchups8.insert(loc = 100, column = 'impact_of_defenders_pts', 
                 value = matchups8.groupby(['PLAYER_ID_x'])['individual_defender_impact_pts'].transform('sum'))

In [ ]:
#Find predicted values
matchups8.insert(loc = 1, column = 'predicted_fanduel', 
                 value = matchups8['impact_of_defenders_fanduel'] + (matchups8['baseline_fanduel']*matchups8['poss_per_game']))
matchups8.insert(loc = 1, column = 'predicted_draftkings', 
                 value = matchups8['impact_of_defenders_draftkings'] + (matchups8['baseline_draftkings']*matchups8['poss_per_game']))

Now I'll push the predicted outcomes to a csv for comparison with observed values

In [ ]:
matchups8.to_csv(f"C:\\Users\gsteele\Other\predicted_values\\{todays_date}.csv")

#matchups8['predicted_draftkings']-matchups8['draftkings']
out = (matchups8['predicted_fanduel'] - matchups8['fanduel']).sort_values().drop_duplicates()
out
#((matchups8['baseline_fanduel']*matchups8['poss_per_game']) - matchups8['fanduel']).round(2).sort_values()

#Isolating the problem in the calculation of 'impact_of_defenders_fanduel/draftkings'
#matchups8[matchups8['impact_of_defenders_fanduel'] < 0]#['OFF_PLAYER_NAME','DEF_PLAYER_NAME','MATCHUP_fanduel_diff','PARTIAL_POSS']]
#(matchups8['fanduel_poss']-matchups8['baseline_fanduel']).sort_values().drop_duplicates()
#matchups7['Season_x'].value_counts()
#(matchups8['predicted_fanduel'] - matchups8['fanduel']).drop_duplicates().sort_values()
matchups8.head(10)

#This appears to be producing inacurate results in the "(matchups8['baseline_fanduel']*matchups8['poss_per_game']))" portion. 
#Need to review both of these values, and/or use simple season average + defensive effect 

matchups8.insert(loc = 1, column = 'predicted_fanduel', 
                 value = matchups8['impact_of_defenders_fanduel'] + (matchups8['baseline_fanduel']*matchups8['poss_per_game']))
matchups8.insert(loc = 1, column = 'predicted_draftkings', 
                 value = matchups8['impact_of_defenders_draftkings'] + (matchups8['baseline_draftkings']*matchups8['poss_per_game']))

# game_counter is producing errant values for baseline_fanduel and baseline_draftkings

**ONE** problem is that game_counter is taking only data from teams playing today, so players who played for a team last year that isn't playing today are having their previous season data dropped. This is not **the** problem which is causing the values to be too high. Cell below demonstrates that fantasy_poss values are much lower for 2019-20 than for 2018-19.  

# Scraping and Cleaning Data

def matchupteam(teamid,season):
    headers = {
'Host': 'stats.nba.com',
'Connection': 'keep-alive',
'Accept': 'application/json, text/plain, */*',
'x-nba-stats-token': 'true',
'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36',
'x-nba-stats-origin': 'stats',
'Referer': f"https://stats.nba.com/team/{teamid}/matchups/",
'Accept-Encoding': 'gzip, deflate, br',
'Accept-Language': 'en-US,en;q=0.9'}

    url = f"https://stats.nba.com/stats/leagueseasonmatchups?DateFrom=&DateTo=&DefTeamID={teamid}&LeagueID=00&Outcome=&PORound=0&PerMode=Totals&Season={season}&SeasonType=Regular+Season"""
    r = requests.get(url, headers=headers).json()
    df = pd.DataFrame(r['resultSets'][0]['rowSet'], columns = r['resultSets'][0]['headers'])
    df.to_csv(f"C:\\Users\gsteele\Other\matchups\\{season}_matchups_{teamid}.csv")

for season in ('2017-18','2018-19','2019-20'):
    for teamid in range (1610612737,1610612768):
        time.sleep(2.3)
        matchupteam(teamid,season)

path = r'C:\Users\gsteele\Other\matchups'
all_files = glob.glob(path + "/*.csv")

csv_list = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    csv_list.append(df)

df2 = pd.concat(csv_list, axis=0, ignore_index=True)

df2.to_csv("C:\\Users\gsteele\Other\matchups\\all_matchups.csv", index=False, encoding='ascii')

#Import
new_format = pd.read_csv(r'C:\Users\gsteele\other\matchups\all_matchups.csv', low_memory = False)
old_format = pd.read_csv(r'C:\users\gsteele\Other\old_matchups.csv', low_memory = False)

#Clean old_format
old_format_df = old_format[(old_format['SEASON_ID'] == '2013-14')
                          | (old_format['SEASON_ID'] == '2014-15')
                          | (old_format['SEASON_ID'] == '2015-16')
                          | (old_format['SEASON_ID'] == '2016-17')
                          ]

#Create/rename columns to match new_format
old_format_df.insert(loc = 1, column = 'PARTIAL_POSS', value = old_format_df['POSS'])
old_format_df.insert(loc = 1, column = 'MATCHUP_AST', value = old_format_df['AST'])
old_format_df.insert(loc = 1, column = 'MATCHUP_TOV', value = old_format_df['TOV'])
old_format_df.insert(loc = 1, column = 'MATCHUP_BLK', value = old_format_df['BLK'])
old_format_df.insert(loc = 1, column = 'MATCHUP_FGM', value = old_format_df['FGM'])
old_format_df.insert(loc = 1, column = 'MATCHUP_FGA', value = old_format_df['FGA'])
old_format_df.insert(loc = 1, column = 'MATCHUP_FG_PCT', value = old_format_df['FG_PCT'])
old_format_df.insert(loc = 1, column = 'MATCHUP_FG3M', value = old_format_df['FG3M'])
old_format_df.insert(loc = 1, column = 'MATCHUP_FG3A', value = old_format_df['FG3A'])
old_format_df.insert(loc = 1, column = 'MATCHUP_FG3_PCT', value = old_format_df['FG3_PCT'])
old_format_df.insert(loc = 1, column = 'HELP_FGM', value = np.nan)
old_format_df.insert(loc = 1, column = 'HELP_FGA', value = np.nan)
old_format_df.insert(loc = 1, column = 'HELP_FG_PERC', value = np.nan)
old_format_df.insert(loc = 1, column = 'MATCHUP_FTM', value = old_format_df['FTM'])
old_format_df.insert(loc = 1, column = 'MATCHUP_FTA', value = (old_format_df['SFL']*2))
old_format_df.insert(loc = 1, column = 'GP', value = np.nan)
old_format_df.insert(loc = 1, column = 'MATCHUP_MIN', value = (old_format_df['POSS'].astype(int)/2))

#Order columns correctly
new_df = new_format.drop('Unnamed: 0',axis=1)
old_df = old_format_df[['SEASON_ID','OFF_PLAYER_ID','OFF_PLAYER_NAME','DEF_PLAYER_ID','DEF_PLAYER_NAME','GP','MATCHUP_MIN',
                       'PARTIAL_POSS','PLAYER_PTS','TEAM_PTS','MATCHUP_AST','MATCHUP_TOV','MATCHUP_BLK','MATCHUP_FGM',
                        'MATCHUP_FGA','MATCHUP_FG_PCT','MATCHUP_FG3M','MATCHUP_FG3A','MATCHUP_FG3_PCT',
                       'HELP_BLK','HELP_FGM','HELP_FGA','HELP_FG_PERC','MATCHUP_FTM','MATCHUP_FTA','SFL']]
data_list = [new_df,old_df]

#Concatenate, then push out csv
matchup_data = pd.concat(data_list)
matchup_data.to_csv(r'c:\users\gsteele\Other\matchup_data.csv')